# Dependendencies

In [4]:
import matplotlib.pyplot as plt
import random as rd
import igraph as ig
import os
import sys
newPath = os.path.dirname(os.path.dirname(os.path.abspath("")))
if newPath not in sys.path:
    sys.path.append(newPath)
from BI.Main.main import bi
import jax
m = bi()

jax.local_device_count 16


# Functions

In [5]:
def random_seed():
    return rd.randint(0,100000000)
random_seed()

10116760

In [6]:
@jax.jit
def scale(x):
    return (x - x.mean()) / x.std() # for 

In [7]:
def sim_covNV(N, T, alpha, beta, epsilon, print = False):

    id_cov = m.dist.normal(0.20, 0.5, sample = True, shape=(N,), seed = random_seed())
    cov_NV = m.dist.normal(alpha+ beta*id_cov,epsilon,sample = True, shape = (T,), seed = random_seed())
    if print: 
        colors = cm.viridis(np.linspace(0, 1, N))  # N distinct colors
        plt.figure(figsize=(12, 6))
        for i in range(cov_NV.shape[0]):
            plt.scatter(np.arange(T), cov_NV[:, i],)

        plt.xlabel("Time")
        plt.ylabel("cov_NV")
        plt.title("cov_NV values over time per individual (colored by row)")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
        plt.tight_layout()
        plt.show()
    return cov_NV.T # rows =individuals, cols = time

In [8]:
def sim_covDF(N, alpha = 0.1, beta = 0.25, epsilon = 0.03):

    id_cov = m.dist.normal(0.20, 0.5, sample = True, shape=(N,), seed = random_seed())
    cov_NV = m.dist.normal(alpha+ beta*id_cov,epsilon,sample = True, shape = (N,), seed = random_seed())
    cov_NV = cov_NV.at[jnp.diag_indices(N)].set(0)
    return cov_NV # rows =individuals, cols = time
#sim_covDF(N = 10)

In [9]:
def grp_belonging_to_mat(block):
    blockMatrix = (block[:, None] == block[None, :]).astype(jnp.int32)
    blockMatrix = blockMatrix.at[jnp.diag_indices(blockMatrix.shape[0])].set(0)
    return blockMatrix

In [10]:
import jax.numpy as jnp
import jax.random as jr
from jax import vmap
import numpy as np
jnp.set_printoptions(precision=2,linewidth=200)

def symmetrize(array):
    return (array + array.T) / 2
    
def sim_grp_equal_prob(G, N):
    # Create a 1D array of ones with length G
    ones = jnp.ones((G,))

    # Normalize to get equal probabilities
    equal_probs = ones / G

    # Generate multinomial probabilities
    grp_prob = m.dist.multinomialprobs(equal_probs, 1, sample=True, shape=(N,))

    # Use vmap with jnp.argmax to sample indices
    return vmap(jnp.argmax)(grp_prob)

def sim_grp(N, Ngrp=2,gByGrp=jnp.array([1,3])):
    return [sim_grp_equal_prob(gByGrp[i],N) for i in range(len(gByGrp))]

def create_random_network(n, rate = 0.2, seed=0):
    """
    Create a random network adjacency matrix using the Erdős-Rényi model.
    
    Parameters:
        n (int): Number of nodes in the network.
        rate (float): ate parameter (mean of the distribution), must be >= 0.
        seed (int): Random seed for reproducibility.
        
    Returns:
        jax.numpy.ndarray: Adjacency matrix of the generated random network.
    """
    # Set the random seed for reproducibility
    key = jax.random.PRNGKey(seed)
    
    upper_tri = jax.random.poisson(key,lam = rate,  shape = (n, n))
    lower_tri = upper_tri
    
    # Make the matrix symmetric to represent an undirected graph
    m = upper_tri + lower_tri
    m = m.at[jnp.diag_indices(m.shape[0])].set(0)
    return m

def viz_network(network, block1 = None):
    
    # Convert the input data to NumPy arrays
    adj_matrix_np = np.array(network)
    if block1 is not None:
        block1_np = np.array(block1)
    else:
        block1_np = None


    # Create an igraph graph from the adjacency matrix
    G = ig.Graph.Adjacency((adj_matrix_np > 0).tolist())

    # Add weights to the edges if the adjacency matrix contains weights
    G.es['weight'] = adj_matrix_np[adj_matrix_np.nonzero()] / 4

    # Define the layout
    layout = G.layout_kamada_kawai()
    scale_factor = 20
    layout = [tuple(coord * scale_factor for coord in point) for point in layout]

    # Map the color array to specific colors
    def create_color_map(n):
        color_list = ['red', 'green', 'blue', 'yellow', 'purple', 'orange', 'cyan', 'magenta', 'brown', 'pink']
        if n > len(color_list):
            raise ValueError(f"Only {len(color_list)} colors available, but got n={n}")

        return {i: color_list[i] for i in range(n)}
    color_map = create_color_map(np.unique(block1_np).size)

    if block1_np is not None:
        node_colors = [color_map[block] for block in block1_np]
    else:
        node_colors = ['gray'] * len(G.vs)

    edge_colors = []
    for edge in G.es:
        # 'edge.source' gives the emitter node index for directed graphs.
        edge_colors.append(node_colors[edge.source])
    #G.es["color"] = edge_colors

    # Plot the graph
    fig, ax = plt.subplots(figsize=(10, 10)) 
    ig.plot(G, layout=layout, 
            vertex_size=10, vertex_color=node_colors, 
            edge_width=G.es['weight'], target=ax,
            #edge_color=G.es["color"],
            edge_arrow_size=4, edge_curved = 0.2)

    # Display the plot
    plt.show()

def create_covariates(N=50, times=00,
b_ij_mean1 = -2, b_ij_sd1 = 0.2, b_ii_mean1 = 0.1, b_ii_sd1 = 0.01,
b_ij_mean = 0.2, b_ij_sd = 0.1, b_ii_mean = 2, b_ii_sd = 0.5,
s_mu= 0.4, s_sd = 0.1, r_mu= 0.1, r_sd= 0.01,
dr_mu = 0.6, dr_sd = 0.1,
Ngrp = 3, print_network = False, add_covNF = True, add_covNV = True):

    # Covariates--------------------------------------
    covNF = scale(m.dist.normal(0,1, sample = True, shape = (1,N), seed = random_seed()))
    covNV = sim_covNV(N, times, 0.25, 0.30, 0.15)
    covDF = sim_covDF(N)
    # Network-----------------------------------------------------------
    ## Block ---------------------------------------
    blocks = sim_grp(N, Ngrp=2,gByGrp=jnp.array([1,Ngrp]))
    block0=blocks[0]
    block1=blocks[1]
    B0 = m.net.block_model(block0,1, b_ij_mean = b_ij_mean1, b_ij_sd = b_ij_sd1, b_ii_mean = b_ii_mean1, b_ii_sd = b_ii_sd1,sample=True)
    B1 = m.net.block_model(block1,Ngrp, b_ij_mean = b_ij_mean, b_ij_sd = b_ij_sd, b_ii_mean = b_ii_mean, b_ii_sd = b_ii_sd,sample=True)
    ### SR shape =  N individuals---------------------------------------
    if add_covNF == True and add_covNV == True:
        focal_individual_predictors = covNF
        focal_individual_predictors = jnp.concat([covNF, jnp.expand_dims(covNV[:,0],0)])
    elif add_covNF == True and add_covNV == False:
        focal_individual_predictors = covNF
    elif add_covNF == False and add_covNV == True:
        focal_individual_predictors = jnp.expand_dims(covNV[:,0],0)
    
    target_individual_predictors=focal_individual_predictors
    sr =  m.net.sender_receiver(focal_individual_predictors,target_individual_predictors,
                                 s_mu= s_mu, s_sd = s_sd, r_mu= r_mu, r_sd= r_sd,
                                 sample=True)
    ## Dyadic shape = N dyads-------------------------------------- 
    dyadic_predictors = m.net.mat_to_edgl(covDF)
    dr = m.net.dyadic_effect(dyadic_predictors=dyadic_predictors,dr_mu = dr_mu, dr_sd = dr_sd, shape=(N,), sample=True)
    
    network=m.net.edgl_to_mat(m.dist.binomial(total_count = jnp.array([10]), logits = B0 + B1 + sr + dr, sample=True ), N)
    network = network/10

    if print_network:
        viz_network(network, block1)

    # Covariates-----------------------------------------------------------
    time = jnp.arange(0,times)
    covF = focal_individual_predictors # Fixed covariate
    jnp.tile(covF[0,:], (times, 1)).T.shape

    # Time-varying covariate

    covDV = create_random_network(N, rate=0.3, seed=10)
    covDV = vmap(lambda x: covDV + x)(time).transpose(1,2,0) # Time-varying dyadic covariate
    covDF = jnp.stack([create_random_network(N, rate=0.8)] * times, axis=2)
    #covDF = jnp.stack([grp_belonging_to_mat(block1)] * times, axis=2)
    return {
        'covF': covNF[0,:],
        'covV': covNV,
        'network': network,
        'covDF': covDF,
        'covDV': covDV,
        'covDF2': grp_belonging_to_mat(block1)
    }

def heatmap(array):
    # Convert to NumPy for Matplotlib
    data_np = jnp.array(array)  # Alternatively, use `data.to_numpy()`

    # Plot heatmap
    plt.imshow(data_np, cmap="viridis", aspect="auto")
    plt.colorbar(label="Value")
    plt.show()


In [11]:
def create_covariates2(N=50, times=00,
b_ij_mean1 = -2, b_ij_sd1 = 0.2, b_ii_mean1 = 0.1, b_ii_sd1 = 0.01,
b_ij_mean = 0.2, b_ij_sd = 0.1, b_ii_mean = 2, b_ii_sd = 0.5,
s_mu= 0.4, s_sd = 0.1, r_mu= 0.1, r_sd= 0.01,
dr_mu = 0.6, dr_sd = 0.1,
Ngrp = 3, print_network = False, add_covNF = True, add_covNV = True,
exposure_alpha = 0.001, exposure_beta= 0.001,
exposure_sigma = 0.001, exposure_max = 10,
print_exposure = True):
    # Covariates--------------------------------------
    covNF = scale(m.dist.normal(0,1, sample = True, shape = (1,N), seed = random_seed()))
    covNV = sim_covNV(N, times, 0.25, 0.30, 0.15)
    covDF = sim_covDF(N)

    # Network-----------------------------------------------------------
    ## Block ---------------------------------------
    blocks = sim_grp(N, Ngrp=2,gByGrp=jnp.array([1,Ngrp]))
    block0=blocks[0]
    block1=blocks[1]
    B0 = m.net.block_model(block0,1, b_ij_mean = b_ij_mean1, b_ij_sd = b_ij_sd1, b_ii_mean = b_ii_mean1, b_ii_sd = b_ii_sd1,sample=True)
    B1 = m.net.block_model(block1,Ngrp, b_ij_mean = b_ij_mean, b_ij_sd = b_ij_sd, b_ii_mean = b_ii_mean, b_ii_sd = b_ii_sd,sample=True)
    ### SR shape =  N individuals---------------------------------------
    if add_covNF == True and add_covNV == True:
        focal_individual_predictors = covNF
        focal_individual_predictors = jnp.concat([covNF, jnp.expand_dims(covNV[:,0],0)])
    elif add_covNF == True and add_covNV == False:
        focal_individual_predictors = covNF
    elif add_covNF == False and add_covNV == True:
        focal_individual_predictors = jnp.expand_dims(covNV[:,0],0)
    
    target_individual_predictors=focal_individual_predictors
    sr =  m.net.sender_receiver(focal_individual_predictors,target_individual_predictors,
                                 s_mu= s_mu, s_sd = s_sd, r_mu= r_mu, r_sd= r_sd,
                                 sample=True)
    
    ## Dyadic shape = N dyads-------------------------------------- 
    dyadic_predictors = m.net.mat_to_edgl(covDF)
    dr = m.net.dyadic_effect(dyadic_predictors=dyadic_predictors,dr_mu = dr_mu, dr_sd = dr_sd, shape=(N,), sample=True)
    

    # Biase of observations based on individual covariates
    exposure = exposure_bias(
        cov = covNF,
        exposure_alpha = exposure_alpha, exposure_beta = exposure_beta,  exposure_sigma = exposure_sigma, exposure_max = exposure_max,
        print_exposure = print_exposure
    )
    
    network_no_bias = m.net.edgl_to_mat(m.dist.binomial(total_count = exposure_max, logits = B0 + B1 + sr + dr, sample=True ), N)
    network_no_bias = network_no_bias/exposure_max
   
    exposure_mat = exposure_vector_to_exposure_matrice(exposure)
    exposure_edgl = m.net.mat_to_edgl(exposure_mat)
    network_bias = m.net.edgl_to_mat(m.dist.binomial(total_count = exposure_edgl, logits = B0 + B1 + sr + dr, sample=True ), N)
    network_bias_old = network_bias
    network_bias = network_bias/exposure_mat
    network_bias = network_bias.at[jnp.diag_indices_from(network_bias)].set(0)
    
    if jnp.any(jnp.isnan(network_bias)):
        nan_indices = jnp.where(jnp.isnan(network_bias))
        print(network_bias_old[nan_indices])
        print(exposure_mat[nan_indices])
        network_bias = network_bias.at[nan_indices].set(0)
    
    if print_network:
        viz_network(network_no_bias, block1)
        viz_network(network_bias, block1)

    # Covariates-----------------------------------------------------------
    time = jnp.arange(0,times)
    covF = focal_individual_predictors # Fixed covariate
    jnp.tile(covF[0,:], (times, 1)).T.shape

    # Time-varying covariate

    covDV = create_random_network(N, rate=0.3, seed=10)
    covDV = vmap(lambda x: covDV + x)(time).transpose(1,2,0) # Time-varying dyadic covariate
    covDF2 = grp_belonging_to_mat(block1)
    plt.show()
    return {
        'covF': covNF[0,:],
        'covV': covNV,
        'network': network_no_bias,
        'networkBias' : network_bias,
        'covDF': covDF,
        'covDV': covDV,
        'covDF2': block1,
        'exposure': exposure
    }


In [12]:
def exposure_bias(cov = m.dist.normal(0,1,sample = True, shape = (10,)),
        exposure_alpha = 0.001, exposure_beta= 0.001,
        exposure_sigma = 0.001, exposure_max = 50,
        print_exposure = True):

    exposure = m.dist.normal(exposure_alpha + exposure_beta*cov ,exposure_sigma, sample = True )
    true_exposure = m.dist.binomial(total_count=exposure_max, probs = m.link.inv_logit(exposure),  sample = True) 
    print(true_exposure)
    if print_exposure:
        plt.scatter(cov,true_exposure)
        plt.xlabel('covariate')
        plt.ylabel('Exposure')
    return true_exposure

def exposure_vector_to_exposure_matrice(exposure_vector):
    mat =  jnp.tile(exposure_vector,(exposure_vector.shape[0],1)) + jnp.tile(exposure_vector,(exposure_vector.shape[0],1)).T
    mat =  mat.at[jnp.diag_indices(exposure_vector.shape[0])].set(0)
    return mat       

# Simulation

In [15]:
N=100 # Number of individuals
times = 50
covariates_data = create_covariates2(N=N, times=times, print_network= False,Ngrp = 3,
                                b_ij_mean1 = 0, b_ij_sd1 = 0.2, b_ii_mean1 = 0.06, b_ii_sd1 = 0.01,
                                b_ij_mean = 0.1, b_ij_sd = 0.05, b_ii_mean = 0.4, b_ii_sd = 0.1,
                                s_mu= .6, s_sd = 0.2, r_mu= 0.6, r_sd= 0.2,
                                dr_mu = .21, dr_sd = 0.1,
                                add_covNF = True, add_covNV = False,
                                exposure_alpha = .0, exposure_beta= .4,
                                exposure_sigma = 0., exposure_max = 10,
                                print_exposure = False
)


[[3 6 3 5 5 6 5 3 5 5 4 5 5 6 4 6 4 3 4 1 8 4 5 5 2 8 4 7 6 4 5 1 6 6 5 2 6 5 4 5 2 5 4 5 6 5 3 6 6 4 5 4 3 4 4 4 6 6 4 3 6 7 6 4 3 6 5 8 8 3 4 8 6 7 4 4 7 7 5 4 7 5 7 6 5 4 4 4 6 2 6 5 5 5 6 6 4 4
  5 6]]


# Model

In [14]:
N_id = 10
a = m.dist.normal(5, 2,  sample = True)
b = m.dist.normal(-1, 0.5,  sample = True)
m.bnn.cov(10,N_id,a,b, sample = True)

Array([[ 3.85,  0.1 ],
       [ 3.54, -1.51],
       [ 4.88,  2.45],
       [ 2.71, -0.96],
       [ 3.07, -0.47],
       [ 5.12, -2.46],
       [ 6.47,  1.89],
       [ 4.02, -0.33],
       [ 2.9 ,  1.56],
       [ 7.28, -3.05]], dtype=float64)

In [ ]:
@jax.jit
def logit(x):
    return jnp.log(x / (1 - x))

def SRM_FULL(
        result_outcomes, 
        dyadic_predictors, focal_individual_predictors, target_individual_predictors, 
        Any, block,
        exposure_mat
        ):

    # SRM + Block model -------------------------------------------------------------
    # Block ---------------------------------------
    B_any = m.net.block_model(Any,1)
    B_block = m.net.block_model(block,5)

    ## SR shape =  N individuals---------------------------------------
    sr =  m.net.sender_receiver(focal_individual_predictors,target_individual_predictors)

    # Dyadic shape = N dyads--------------------------------------  
    dr = m.net.dyadic_effect(dyadic_predictors)

    m.dist.binomial(total_count = m.net.mat_to_edgl(exposure_mat), logits = B_any + B_block + sr + dr, obs = result_outcomes, name= 'latent network' )
 

In [ ]:
multiplex = [SRM_data["network"],SRM_data["network"],SRM_data["network"]]
multiplex_exposure = [SRM_data["exposure_mat"],SRM_data["exposure_mat"],SRM_data["exposure_mat"]]
# TODO : mask to handle NA

In [47]:
focal_individual_predictors = SRM_data["focal_individual_predictors"]
target_individual_predictors = SRM_data["target_individual_predictors"]
focal_individual_predictors
K = len(multiplex)

# FIxed effect for SR
fixed_sr = []
for k in range(K):
    sr_ff, focal_effects, target_effects = m.net.nodes_terms(focal_individual_predictors,target_individual_predictors, N_var   = 1, s_mu = 0, s_sd = 1, r_mu = 0, r_sd = 1, sample = True, diag = False  ) # focal_name = 'focal_effects' + k
    fixed_sr.append(sr_ff)
fixed_sr[0].shape

(100, 2)

In [43]:
# RF effect for SR
N_id = 100
sample = True
sr_raw =  m.dist.normal(0, 1, shape=(2*K, N_id), name = 'sr_raw', sample = sample)
sr_sigma =  m.dist.exponential(1, shape= (2*K,), name = 'sr_sigma', sample = sample)
sr_L = m.dist.lkjcholesky(2*K, 2, name = "sr_L", sample = sample)
rf = (sr_L @ sr_raw).T * sr_sigma
rf.shape # col1 = focal effect fo r layer 1, col2 = same for layer 2, col 3 = same for layer 3, col4 = receiver effect for layer 1, etc...

(100, 6)

In [52]:
# FIxed effect for dyads
dyadic_predictors = SRM_data['dyadic_predictors']
dyad_effects = []
for k in range(K):
    dyad_effects.append(m.dist.normal(0, 1, name= 'dyad_effects'+str(k), shape = (dyadic_predictors.ndim - 1,), sample = sample, seed = k))

dyad_effects

[Array([-0.21], dtype=float64),
 Array([-1.18], dtype=float64),
 Array([-0.19], dtype=float64)]

In [61]:
# RF for dyads
N_dyads = 100*99
dr_raw =  m.dist.normal(0, 1, shape=(2*K,N_dyads), name = 'dr_raw', sample = sample)
dr_sigma = m.dist.exponential(1, shape=(K,), name = 'dr_sigma', sample = sample )
dr_L = m.dist.lkjcholesky(2*K, 2, name = 'dr_L', sample = sample)
dr_rf = (dr_L @ dr_raw).T * jnp.repeat(dr_sigma, 2)
dr_rf.shape# col1 = col1 of layer 1, col2 =  col1 of layer 2, col 3 =  col1 of layer 3, col4 =  col2 of layer 1, etc...

(9900, 6)

In [81]:
D_corr = dr_L @ dr_L.T
"""
    for(m in 1:(N_responses-1)){
    for(n in (m+1):N_responses){
     target += normal_lpdf(D_corr[m+N_responses, n+N_responses] | D_corr[m, n],   bandage_penalty);
     target += normal_lpdf(D_corr[m, n+N_responses]   | D_corr[n, m+N_responses], bandage_penalty);
    }}
"""
m = bi()
bandage_penalty = 0.01 # penalty =  D_corr[m, n]
for f in range(K-1):
    for n in range(f+1, K):       
        m.dist.normal(D_corr[f+K, n+K], bandage_penalty,  seed = 1, obs = D_corr[f, n], name = 'D_corr2' )
        m.dist.normal(D_corr[f, n+k], bandage_penalty,  seed = 1, obs = D_corr[n, f+k], name = 'D_corr3' )


jax.local_device_count 16


In [17]:
@jax.jit
def logit(x):
    return jnp.log(x / (1 - x))

def SRM_detailed(
        result_outcomes, 
        dyadic_predictors, 
        focal_individual_predictors, 
        target_individual_predictors, 
        Any, block,
        exposure_mat
        ):

    # SRM + Block model -------------------------------------------------------------
    # Block ---------------------------------------
    B_any = m.net.block_model(Any,1)
    B_block = m.net.block_model(block,5)

    ## SR shape =  N individuals---------------------------------------
    sr =  m.net.sender_receiver(focal_individual_predictors,target_individual_predictors)

    # Dyadic shape = N dyads--------------------------------------  
    dr_ff, dyad_effects = Neteffect.dyadic_terms(dyadic_predictors, d_m = d_m, d_sd = d_sd, sample = sample)
    dr_rf, dr_raw, dr_sigma, dr_L =  Neteffect.dyadic_random_effects(
        dr_ff.shape[0], dr_mu = dr_mu, dr_sd = dr_sd, dr_sigma = dr_sigma, 
        cholesky_dim = cholesky_dim, cholesky_density = cholesky_density, sample = sample)
    dr = dr_ff + dr_rf

    m.dist.binomial(total_count = m.net.mat_to_edgl(exposure_mat), logits = B_any + B_block + sr + dr, obs = result_outcomes, name= 'latent network' )
 

In [18]:
m = bi()
m.data_on_model = dict(
    Any = jnp.zeros_like(covariates_data['covDF2']), # array of one of shape(1,N)
    block = covariates_data['covDF2']-1, # array of shape(1,N) representing node group belonging
    result_outcomes = m.net.mat_to_edgl(covariates_data['networkBias']), 
    dyadic_predictors = m.net.mat_to_edgl(covariates_data['covDF']),
    focal_individual_predictors =  covariates_data['covF'].reshape(1,covariates_data['covF'].shape[0]),
    target_individual_predictors =  covariates_data['covF'].reshape(1,covariates_data['covF'].shape[0]),
    exposure_mat = exposure_vector_to_exposure_matrice(covariates_data['exposure'][0,:])
)

m.fit(SRM_FULL) 
m.summary()

jax.local_device_count 16


NameError: name 'SRM_FULL' is not defined

In [19]:
SRM_data = {}
SRM_data['network'] = m.net.mat_to_edgl(covariates_data['networkBias'])
SRM_data['block_intercept'] = jnp.zeros_like(covariates_data['covDF2'])
SRM_data['block_cov'] = m.net.mat_to_edgl(covariates_data['covDF'])
SRM_data['focal_individual_predictors'] =  covariates_data['covF'].reshape(1,covariates_data['covF'].shape[0])
SRM_data['target_individual_predictors'] =  covariates_data['covF'].reshape(1,covariates_data['covF'].shape[0])
SRM_data['dyadic_predictors'] = m.net.mat_to_edgl(covariates_data['covDF'])
SRM_data['exposure_mat'] = exposure_vector_to_exposure_matrice(covariates_data['exposure'][0,:])
